In [13]:
# -*- coding: utf-8 -*-
"""
Created on Wed Apr 22 05:42:28 2020

@author: paras
"""

import pandas as pd
import numpy as np
from sklearn.externals import joblib 

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

df = pd.read_csv("Churn_Modelling.csv")

df_x = df.iloc[:, 3:13]
df_y = df.iloc[:, 13]

# df.isna().sum()

In [14]:
def clean_data(df):

    # le = LabelEncoder()
    # df_x.iloc[:, 2] = le.fit_transform(df_x.iloc[:, 2])

    # le.fit(df_x.Geography.drop_duplicates()) 
    # df_x.Geography = le.transform(df_x.Geography)

    df = pd.get_dummies(data = df, drop_first = False)
    df.drop(["Gender_Female"], axis=1, inplace=True)
    df.rename(columns={"Gender_Male": "Gender"}, inplace=True)
    return df

In [19]:
clean_data(df_x).head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain,Gender
0,619,42,2,0.00,1,1,1,101348.88,1,0,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,0,1,0
2,502,42,8,159660.80,3,1,0,113931.57,1,0,0,0
3,699,39,1,0.00,2,0,0,93826.63,1,0,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,0,1,0


In [28]:
df_one = pd.DataFrame(df_x.iloc[2, :])
df_one.reset_index(drop=True)
df_one

,2
CreditScore,502
Geography,France
Gender,Female
Age,42
Tenure,8
Balance,159661
NumOfProducts,3
HasCrCard,1
IsActiveMember,0
EstimatedSalary,113932


In [24]:
clean_data(df_one).head()

KeyError: "['Gender_Female'] not found in axis"

In [ ]:
columnTransformer = ColumnTransformer([('encoder', OneHotEncoder(), [1])], remainder='passthrough')
col_tnf = columnTransformer.fit_transform(df_x)
df_x = np.array(col_tnf, dtype = np.str)
df_x = df_x[:, 1:]



from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_x, df_y, test_size = 0.2, random_state = 0)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_test

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense

classifier = Sequential()
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)


y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
accuracy = (cm[0,0]+cm[1, 1])/(cm[0,0]+cm[1, 1]+cm[1,0]+cm[0, 1])

joblib.dump(classifier, 'prediction_classifier.pkl') 

